## Getting started with OpenAI Assistants API

This is a quick start tutorial to get started with using [Assistants API](https://platform.openai.com/docs/assistants/overview).

### Environment Setup

In [ ]:
!pip install openai
!pip install python-dotenv

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import time

### OpenAI API Setup

The OpenAI API key is placed in a `.env` file. If you don't have one, create a file named `.env` and update with the following. Replace `sk_this_is_a_sample_key` with your OpenAI API key.

`OPENAI_API_KEY=sk_this_is_a_sample_key`

In [2]:
_ = load_dotenv()

client = OpenAI(
    api_key = os.environ['OPENAI_API_KEY']
)

### Create an assistant

In [3]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

### Create a thread

In [4]:
thread = client.beta.threads.create()

### Add a message to the thread

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to add all numbers from 1 to 100. Can you help me with that and also provide the formula to do that?"
)

### List the messages in the thread

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json())

### Run the assistant

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [ ]:
run.status

In [ ]:
updated_run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

updated_run.status

### Display the message

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [ ]:
messages.data[0].content[0].text.value

### Utility functions for polling and displaying assistant response

In [ ]:
# The function to poll the run status in a regular interval for a specified amount of times
def get_assistant_response(thread_id, run_id, callback, interval=2, max_attempts=60):
    attempts = 0
    while attempts < max_attempts:
        try:
            updated_run = client.beta.threads.runs.retrieve(
              thread_id=thread_id,
              run_id=run_id
            )
            status = updated_run.status
            # Check if the status indicates completion
            if status == "completed":
                callback(status, thread_id)
                break
            else:
                callback(f"Run status: {status}")
                
            time.sleep(interval)
            attempts += 1   
        except Exception as e:
            callback(f"Error: {str(e)}")
            time.sleep(interval)
            attempts += 1
    else:
        # If max_attempts reached without completion, call the callback with a timeout message
        callback("Timeout: Assistant didn't respond in time. Please try again.")

# Callback function to handle the run status
def handle_response(status, thread_id=None):
    if status == "completed":
        print("Run completed successfully.")
        display_messages(thread_id)
    else:
        print(status)

# Function to display the assistant's response
def display_messages(thread_id):
    messages = client.beta.threads.messages.list(
      thread_id=thread_id
    )
    print(f"Assistants Response: {messages.data[0].content[0].text.value}")

In [ ]:
thread_1 = client.beta.threads.create(
    messages = [
        {
            "role": "user",
            "content": "I need to add all numbers from 1 to 1000. Can you help me with that?"
        }
    ]
)

In [ ]:
run_1 = client.beta.threads.runs.create(
    thread_id=thread_1.id,
    assistant_id=assistant.id
)

In [ ]:
get_assistant_response(thread_1.id, run_1.id, handle_response)

### Messages array

In [ ]:
message_json = json.loads(messages.model_dump_json())
print(json.dumps(message_json, indent=2))

In [ ]:
print(message_json["data"][0]["content"][0]["text"]["value"])